# BioEquity Insight Script Walkthrough

This notebook breaks down the **BioEquity Insight** Python script step by step. Each **Cell N** shows the exact code from our script followed by an explanation of why those choices were made. 

---

## Cell 1: Imports and Theming

```python
import langchain
from langchain_openai import ChatOpenAI       # Changed to use ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import yfinance as yf
import requests
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance
import seaborn as sns
sns.set_theme()

import io
import base64
import pandas as pd
from typing import Any

import os
from dotenv import load_dotenv

import plotly.graph_objects as go
from plotly.subplots import make_subplots
```

### ❓ Why?
- LangChain & ChatOpenAI: structure LLM calls, swap models seamlessly.
- yfinance & requests: fetch stock info and web‑based news.
- datetime & pandas: date handling and DataFrame manipulations.
- matplotlib, mplfinance, seaborn, plotly: mix of static & interactive plotting with fallbacks.
- dotenv & os: load API keys from .env, avoiding hard‑coding secrets.

## Cell 2: Environment Variables

```python
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CX      = os.getenv("GOOGLE_CX")
```

### ❓ Why?
- Securely load OPENAI_API_KEY, GOOGLE_API_KEY, and GOOGLE_CX from environment to keep credentials out of source control.

## Cell 3: Stock Data Retrieval

```python
def get_stock_data(ticker: str) -> tuple:
    """Retrieve basic stock info for the given ticker."""
    stock = yf.Ticker(ticker)
    info = stock.info
    summary = (
        f"Name: {info.get('longName', 'N/A')}\n"
        f"Short Name: {info.get('shortName', 'N/A')}\n"
        f"Symbol: {info.get('symbol', ticker)}\n"
        f"Sector: {info.get('sector', 'N/A')}\n"
        f"Industry: {info.get('industry', 'N/A')}\n"
        f"Currency: {info.get('currency', 'N/A')}\n"
        f"Exchange: {info.get('exchange', 'N/A')}\n"
        f"Current Price: {info.get('regularMarketPrice', 'N/A')}\n"
        f"Open: {info.get('open', 'N/A')}\n"
        f"Previous Close: {info.get('previousClose', 'N/A')}\n"
        f"Day's Low/High: {info.get('dayLow', 'N/A')} / {info.get('dayHigh', 'N/A')}\n"
        f"52-Week Low/High: {info.get('fiftyTwoWeekLow', 'N/A')} / {info.get('fiftyTwoWeekHigh', 'N/A')}\n"
        f"Volume: {info.get('volume', 'N/A')}\n"
        f"Average Volume: {info.get('averageVolume', 'N/A')}\n"
        f"Market Cap: {info.get('marketCap', 'N/A')}\n"
        f"Beta: {info.get('beta', 'N/A')}\n"
        f"PE Ratio (TTM): {info.get('trailingPE', 'N/A')}\n"
        f"EPS (TTM): {info.get('trailingEps', 'N/A')}\n"
        f"Dividend Rate: {info.get('dividendRate', 'N/A')}\n"
        f"Dividend Yield: {info.get('dividendYield', 'N/A')}\n"
        f"Ex-Dividend Date: {info.get('exDividendDate', 'N/A')}\n"
        f"Earnings Timestamp: {info.get('earningsTimestamp', 'N/A')}\n"
        f"1-Year Target Estimate: {info.get('targetMeanPrice', 'N/A')}\n"
        f"Fifty Day Average: {info.get('fiftyDayAverage', 'N/A')}\n"
        f"Two Hundred Day Average: {info.get('twoHundredDayAverage', 'N/A')}\n"
        f"Long Business Summary: {info.get('longBusinessSummary', 'N/A')}\n"
    )

    try:
        today = dt.date.today()
        if today.weekday() == 6:  # Sunday
            days_to_subtract = 2
        elif today.weekday() == 5:  # Saturday
            days_to_subtract = 1
        else:
            days_to_subtract = 0
        most_recent_business_day = today - dt.timedelta(days=days_to_subtract)

        print(f"Fetching complete historical data for {ticker}")
        historical = stock.history(period="max")

        if not historical.empty:
            summary += (
                f"\nHistorical Data available from "
                f"{historical.index[0].strftime('%Y-%m-%d')} to "
                f"{historical.index[-1].strftime('%Y-%m-%d')}\n"
                "Last 5 trading days:\n" +
                historical.tail(5).to_string() + "\n"
            )
        else:
            historical = stock.history(period="1y")
            summary += (
                "\nUsing period-based historical data (last 5 trading days):\n" +
                historical.tail(5).to_string() + "\n"
            )
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        historical = pd.DataFrame()
        summary += "\nCould not retrieve historical data for this ticker.\n"

    return summary, historical
```

### ❓ Why?
- Fetches comprehensive summary plus full history (period="max") with fallback to one year.
- Weekend adjustment ensures only complete trading days are shown.
- Error handling prints failures and returns empty DataFrame gracefully.

## Cell 4: Recent Catalysts via Google Custom Search

```python
def get_recent_catalysts(company_names, api_key: str, cx: str) -> dict:
    """
    Use Google Custom Search API to find recent catalysts for the given companies.
    Returns a mapping from company name to formatted markdown snippets.
    """
    if isinstance(company_names, str):
        company_names = [company_names]

    results = {}
    for company_name in company_names:
        search_query = f"{company_name} recent stock catalysts, FDA approvals, clinical trials, significant news"
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            "q": search_query,
            "key": api_key,
            "cx": cx,
            "num": 5
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            items = response.json().get("items", [])
            catalysts = [
                f"- {item.get('title','No title')}\n  {item.get('snippet','No snippet')}\n  Link: {item.get('link','No link')}"
                for item in items
            ]
            results[company_name] = "\n\n".join(catalysts) if catalysts else "No recent catalysts found for this company."
        except Exception as e:
            results[company_name] = f"Error fetching catalysts: {str(e)}"

    return results
```

### ❓ Why?
- Queries Google Custom Search for catalysts, trials, approvals.
- Limits to top 5 results for brevity.
- Formats results as markdown bulleted lists for easy display.

## Cell 5: Ticker Extraction Prompt Template

```python
ticker_prompt_template = (
    "You are a helpful financial assistant. "
    "Extract the stock ticker symbols (uppercase, 1-5 letters) from the user's request.\n"
    "If the user is comparing or asking about multiple companies, return ALL relevant ticker symbols, separated by commas.\n"
    "If only one ticker is mentioned, just return that single ticker.\n"
    "Only return the ticker symbols, nothing else.\n"
    "For example, if the prompt mentions 'Compare Merck and Pfizer', output 'MRK,PFE'. "
    "For 'How is Merck doing?', output 'MRK'.\n\n"
    "User Prompt: {user_prompt}\n"
    "Tickers:"
)

ticker_prompt = PromptTemplate(
    input_variables=["user_prompt"],
    template=ticker_prompt_template
)
```

### ❓ Why?
- Leverages LLM to handle natural‑language variations in ticker mentions.
- Ensures consistent, comma‑separated uppercase output.

## Cell 6: LLM Initialization & Chains

```python
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,  # Use environment variable
    model="o3-mini",         # Using the standard chat model
)

ticker_chain = LLMChain(llm=llm, prompt=ticker_prompt)
```

### ❓ Why?
- ChatOpenAI replaces deprecated OpenAI class.
- o3-mini chosen for speed/cost balance.
- LLMChain couples prompt + model into a reusable pipeline.

## Cell 7: Ticker Extraction Function

```python
def extract_tickers_from_prompt(user_prompt: Any) -> list:
    """Extract one or more ticker symbols from the user prompt."""
    prompt_text = user_prompt.content if hasattr(user_prompt, "content") else user_prompt
    print("RAW PROMPT ►", repr(prompt_text))

    tickers_str = ticker_chain.run(user_prompt=prompt_text).strip()
    print("RAW TICKERS ►", repr(tickers_str))

    tickers = [ticker.strip().upper() for ticker in tickers_str.split(',')]
    tickers = [ticker for ticker in tickers if ticker]
    print(f"Extracted {len(tickers)} ticker(s): {tickers}")
    return tickers
```

### ❓ Why?
- Supports both chainlit Message objects and raw strings.
- Prints debugging info for prompt and extracted tickers.
- Cleans and uppercases values, removing empties.

## Cell 8: Analysis Prompt & Chain

```python
analysis_prompt_template = (
    "You are a financial analyst. Below is the Yahoo Finance data for the stock and recent catalysts:\n\n"
    "### Stock Data:\n{stock_data}\n\n"
    "### Recent Catalysts:\n{catalysts}\n\n"
    "Based on all this information, answer the following question, but with smart, accurate, numerical analysis and well-thought out takes. Include insights from both the stock data and catalysts in your analysis:\n"
    "{user_prompt}\n"
)

analysis_prompt = PromptTemplate(
    input_variables=["stock_data", "catalysts", "user_prompt"],
    template=analysis_prompt_template
)

analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
```

### ❓ Why?
- Focuses the LLM on two core inputs: raw stock metrics and recent catalysts.
- Encourages numerical rigor and integrated insights.

## Cell 9: Main Orchestration (answer_user_query)

```python
def answer_user_query(user_prompt: str) -> dict:
    try:
        # 1️⃣ Extract ticker(s)
        tickers = extract_tickers_from_prompt(user_prompt)
        print(f"Extracted Tickers: {tickers}")

        if not tickers:
            return {"text": "I couldn't identify any stock tickers in your request. Please mention specific companies or stock symbols."}

        # 2️⃣ Collect data for each ticker
        all_stock_data = {}
        all_historical_data = {}
        company_names = {}

        for ticker in tickers:
            stock_data, historical_data = get_stock_data(ticker)
            all_stock_data[ticker] = stock_data
            all_historical_data[ticker] = historical_data

            lines = stock_data.split('\n')
            company_name = lines[0].replace("Name: ", "").strip() if lines else ticker
            company_names[ticker] = company_name

            # Trim long business summaries
            for i, line in enumerate(lines):
                if line.startswith("Long Business Summary:"):
                    summary_text = line.replace("Long Business Summary: ", "")
                    if len(summary_text) > 300:
                        lines[i] = f"Long Business Summary: {summary_text[:300]}..."
                    break
            all_stock_data[ticker] = '\n'.join(lines)

        # 3️⃣ Fetch recent catalysts
        all_catalysts = get_recent_catalysts(
            list(company_names.values()),
            GOOGLE_API_KEY,
            GOOGLE_CX
        )

        # 4️⃣ Run analysis
        if len(tickers) > 1:
            comparative_prompt = create_comparative_analysis_prompt(
                tickers, all_stock_data, all_catalysts, company_names, user_prompt
            )
            result = llm.invoke(comparative_prompt).content
        else:
            t = tickers[0]
            result = analysis_chain.run(
                stock_data=all_stock_data[t],
                catalysts=all_catalysts[company_names[t]],
                user_prompt=user_prompt
            )

        # 5️⃣ Generate chart script
        chart_script = None
        chart_figure = None

        if len(tickers) > 1:
            chart_script = generate_comparative_chart_script(
                tickers, all_stock_data, all_historical_data, user_prompt
            )
            chart_figure = execute_comparative_chart_script(
                chart_script, all_historical_data, tickers
            )
        else:
            t = tickers[0]
            chart_script = generate_chart_script(
                all_stock_data[t], t, all_historical_data[t], user_prompt
            )
            chart_figure = execute_chart_script(
                chart_script, all_historical_data[t], t
            )

        # 6️⃣ Assemble final response
        sources_text = []
        for ticker, company_name in company_names.items():
            if company_name in all_catalysts:
                sources_text.append(f"### Sources for {company_name} ({ticker}):\n{all_catalysts[company_name]}")

        final_response = f"### Analysis:\n{result}\n\n" + "\n\n".join(sources_text)

        return {
            "text": final_response,
            "chart_figure": chart_figure,
            "script": chart_script
        }

    except Exception as e:
        import traceback
        print(f"Error in answer_user_query: {e}")
        print(traceback.format_exc())
        return {"text": f"An error occurred while processing your request: {e}"}
```

### ❓ Why?
- Stepwise pipeline: clear separation of extraction, data retrieval, analysis, visualization, response assembly.
- Token efficiency: trims lengthy summaries.
- Robustness: graceful failure paths for missing tickers, API errors, or chart generation failures.

## Cell 10: Single‑Ticker Chart Script Generation

```python
def generate_chart_script(stock_data: str, ticker: str, historical_data: pd.DataFrame, user_prompt: str) -> str:
    """
    Ask OpenAI to generate a matplotlib/plotly chart script for the given stock data.
    Returns only Python code for a function called 'generate_chart'.
    """
    chart_prompt = f"""
    Create a Python function that generates a visualization specifically addressing this user question:
    "{user_prompt}"

    Ticker: {ticker}

    Stock Summary Data:
    {stock_data}

    The historical data is available as a pandas DataFrame called 'historical_data' with the following structure:
    {historical_data.head().to_string()}

    Additional data is provided in the 'data_dict' parameter, containing:
    - ticker_info: Dictionary with all Yahoo Finance data
    - key_metrics: Dictionary with important metrics
    - dividend_history: Pandas Series with dividend history
    - dividend_df: DataFrame with 'Dividend' column
    - income_statement, balance_sheet, cash_flow: Financial statement DataFrames
    - recommendations: Analyst recommendations DataFrame
    - major_holders, institutional_holders: Ownership DataFrames

    IMPORTANT NOTES:
    1. Use naive datetime objects (no timezone) for date comparisons.
    2. Use pandas frequency strings 'YE' and 'ME' for year/month end.
    3. Use the datetime module as 'dt'.

    Write a function called 'generate_chart' that:
    1. Takes historical_data as the first parameter and data_dict as the second.
    2. Creates an informative Plotly figure.
    3. Checks for existence of keys/DataFrames before use.
    4. Returns a plotly.graph_objects.Figure object.

    Return ONLY the Python code without any explanation.
    """

    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain

    chart_prompt_template = PromptTemplate(
        input_variables=["chart_prompt"],
        template="{chart_prompt}"
    )
    chart_chain = LLMChain(llm=llm, prompt=chart_prompt_template)

    chart_script = chart_chain.run(chart_prompt=chart_prompt).strip()
    print("=== Generated Chart Script ===")
    print(chart_script)
    print("=== End Chart Script ===\n")
    return chart_script
```

### ❓ Why?
- Offloads charting logic to LLM, returning a ready‑to‑execute function.
- Strict “code only” requirement simplifies downstream execution.

## Cell 11: Executing Generated Single‑Ticker Charts

```python
def execute_chart_script(script: str, historical_data: pd.DataFrame, ticker: str = None) -> object:
    """Execute the generated chart script and return a Plotly figure."""
    try:
        namespace = {
            'pd': pd,
            'plt': plt,
            'np': __import__('numpy'),
            'go': go,
            'make_subplots': make_subplots,
            'plotly': __import__('plotly'),
            'historical_data': historical_data.copy(),
            'ticker': ticker,
            'dt': dt,
            'datetime': dt,
        }

        if hasattr(namespace['historical_data'].index, 'tz') and namespace['historical_data'].index.tz is not None:
            namespace['historical_data'].index = namespace['historical_data'].index.tz_localize(None)

        # Load additional stock data
        if ticker:
            try:
                stock = yf.Ticker(ticker)
                namespace['ticker_info'] = stock.info
                metrics = {}
                key_fields = [
                    'regularMarketPrice', 'previousClose', 'open', 'dayLow', 'dayHigh',
                    'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'volume', 'averageVolume',
                    'marketCap', 'beta', 'trailingPE', 'forwardPE', 'trailingEps',
                    'dividendRate', 'dividendYield', 'exDividendDate',
                    'fiftyDayAverage', 'twoHundredDayAverage',
                    'shortRatio', 'profitMargins', 'operatingMargins'
                ]
                for field in key_fields:
                    if field in stock.info:
                        metrics[field] = stock.info[field]
                namespace['key_metrics'] = metrics

                dividend_history = stock.dividends
                if not dividend_history.empty and hasattr(dividend_history.index, 'tz') and dividend_history.index.tz is not None:
                    dividend_history.index = dividend_history.index.tz_localize(None)
                namespace['dividend_history'] = dividend_history
                namespace['dividend_df'] = dividend_history.to_frame('Dividend') if not dividend_history.empty else pd.DataFrame()

                for financial_stmt, stmt_name in [
                    (stock.income_stmt, 'income_statement'),
                    (stock.balance_sheet, 'balance_sheet'),
                    (stock.cash_flow, 'cash_flow')
                ]:
                    if not financial_stmt.empty and hasattr(financial_stmt.columns, 'tz') and financial_stmt.columns.tz is not None:
                        financial_stmt.columns = financial_stmt.columns.tz_localize(None)
                    namespace[stmt_name] = financial_stmt

                recommendations = stock.recommendations
                if not recommendations.empty and hasattr(recommendations.index, 'tz') and recommendations.index.tz is not None:
                    recommendations.index = recommendations.index.tz_localize(None)
                namespace['recommendations'] = recommendations

                namespace['calendar'] = stock.calendar
                namespace['major_holders'] = stock.major_holders
                namespace['institutional_holders'] = stock.institutional_holders

                print(f"Data provided to chart function:")
                for key, value in namespace.items():
                    if key not in ['pd','plt','np','go','make_subplots','plotly']:
                        data_type = type(value).__name__
                        data_size = value.shape if hasattr(value, 'shape') else (len(value) if isinstance(value, dict) else "N/A")
                        print(f"- {key}: {data_type} ({data_size})")
            except Exception as e:
                print(f"Failed to add financial data: {e}")
                namespace.update({
                    'ticker_info': {}, 'key_metrics': {}, 'dividend_history': pd.Series(),
                    'dividend_df': pd.DataFrame(), 'income_statement': pd.DataFrame(),
                    'balance_sheet': pd.DataFrame(), 'cash_flow': pd.DataFrame()
                })

        exec(script, namespace)

        if 'generate_chart' not in namespace:
            print("Error: generate_chart function not found in script")
            return None

        import inspect
        sig = inspect.signature(namespace['generate_chart'])
        params = list(sig.parameters.keys())

        if len(params) == 1:
            fig = namespace['generate_chart'](historical_data)
        else:
            data_dict = {k: v for k, v in namespace.items()
                         if k not in ['pd','plt','np','go','make_subplots','plotly','generate_chart']}
            fig = namespace['generate_chart'](historical_data, data_dict)

        return fig
    except Exception as e:
        print(f"Error executing chart script: {e}")
        import traceback; traceback.print_exc()
        # Fallback
        try:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=historical_data.index, y=historical_data['Close'], mode='lines', name='Close Price'))
            fig.update_layout(title=f"{ticker} Stock Price", xaxis_title="Date", yaxis_title="Price (USD)")
            return fig
        except:
            return None
```

### ❓ Why?
- Namespace sandboxing for secure exec.
- Loads comprehensive stock info into namespace for LLM‑generated code.
- Detects chart function signature to pass correct arguments.
- Provides simple fallback chart on failure.

## Cell 12A: Comparative Analysis Prompt

```python
def create_comparative_analysis_prompt(tickers, all_stock_data, all_catalysts, company_names, user_prompt):
    """Create a prompt for comparative analysis of multiple stocks."""
    prompt = "You are a financial analyst. Below is the Yahoo Finance data for multiple stocks and their recent catalysts:\n\n"

    # Add stock data for each ticker
    for ticker in tickers:
        company_name = company_names[ticker]
        prompt += f"### Stock Data for {company_name} ({ticker}):\n{all_stock_data[ticker]}\n\n"
        if company_name in all_catalysts:
            prompt += f"### Recent Catalysts for {company_name}:\n{all_catalysts[company_name]}\n\n"

    prompt += (
        "Based on all this information, provide a comparative analysis answering the following question. "
        "Include specific metrics, performance trends, and insights about relative strengths and weaknesses:\n"
        f"{user_prompt}\n"
    )

    return prompt
```

### ❓ Why?
- Mirrors single‑ticker prompt but loops to include each company’s data & catalysts.
- Ensures comparative context for the LLM.

## Cell 12B: Comparative Chart Script Generation

```python
def generate_comparative_chart_script(tickers, all_stock_data, all_historical_data, user_prompt):
    """Generate a chart script for comparing multiple stocks."""
    historical_samples = {}
    for ticker in tickers:
        if not all_historical_data[ticker].empty:
            historical_samples[ticker] = all_historical_data[ticker].head().to_string()

    chart_prompt = f"""
    Create a Python function that generates a comparative visualization specifically addressing this user question:
    "{user_prompt}"

    Tickers being compared: {', '.join(tickers)}

    Stock Summary Data:
    {', '.join([f"{ticker}: {all_stock_data[ticker].split('Long Business Summary')[0]}" for ticker in tickers])}

    The historical data for each ticker is available as separate DataFrames in a dictionary called 'historical_data_dict'.
    Example structure for '{tickers[0]}':
    {historical_samples.get(tickers[0], 'No data available')}

    Write a function called 'generate_chart' that:
    1. Takes historical_data_dict as the first parameter (a dictionary mapping ticker symbols to their historical DataFrame)
    2. Creates a comparative visualization using Plotly (import plotly.graph_objects as go)
    3. Shows metrics and trends SPECIFICALLY RELEVANT to comparing these stocks based on the user's question
    4. Handles cases where data might be missing for some tickers
    5. Uses clear labels, legends, and potentially different colors for each ticker
    6. Returns a plotly.graph_objects.Figure object

    For comparative charts, consider:
    - Normalized price charts (setting all stocks to 100% at a specific start date)
    - Side-by-side metrics comparisons
    - Performance metrics over the same time period

    Return ONLY the Python code without any explanation.
    """

    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain

    chart_prompt_template = PromptTemplate(
        input_variables=["chart_prompt"],
        template="{chart_prompt}"
    )
    chart_chain = LLMChain(llm=llm, prompt=chart_prompt_template)

    try:
        chart_script = chart_chain.run(chart_prompt=chart_prompt).strip()
        print("=== Generated Comparative Chart Script ===")
        print(chart_script)
        print("=== End Comparative Chart Script ===\n")
        return chart_script
    except Exception as e:
        print(f"Error generating comparative chart script: {e}")
        return None
```

### ❓ Why?
- Similar to single‑ticker chart generation but tailored for multiple data series.
- Provides clear instructions for comparative metrics and labeling.

## Cell 12C: Executing Comparative Chart Script

```python
def execute_comparative_chart_script(script, all_historical_data, tickers):
    """Execute the generated comparative chart script and return a Plotly figure."""
    try:
        namespace = {
            'pd': pd,
            'plt': plt,
            'np': __import__('numpy'),
            'go': go,
            'make_subplots': make_subplots,
            'plotly': __import__('plotly'),
            'historical_data_dict': {},
            'dt': dt,
            'datetime': dt,
        }

        for ticker in tickers:
            historical_data = all_historical_data.get(ticker, pd.DataFrame()).copy()
            if not historical_data.empty and hasattr(historical_data.index, 'tz') and historical_data.index.tz:
                historical_data.index = historical_data.index.tz_localize(None)
            namespace['historical_data_dict'][ticker] = historical_data

        exec(script, namespace)
        if 'generate_chart' not in namespace:
            print("Error: generate_chart function not found in script")
            return create_fallback_comparative_chart(all_historical_data, tickers)

        fig = namespace['generate_chart'](namespace['historical_data_dict'])
        return fig

    except Exception as e:
        print(f"Error executing comparative chart script: {e}")
        import traceback; traceback.print_exc()
        return create_fallback_comparative_chart(all_historical_data, tickers)
```

### ❓ Why?
- Sandboxes the comparative script with its own historical_data_dict.
- Falls back to a simple comparison chart on error.

## Cell 12D: Fallback Comparative Chart

```python
def create_fallback_comparative_chart(all_historical_data, tickers):
    """Create a simple fallback chart when the generated script fails."""
    try:
        fig = go.Figure()
        for ticker in tickers:
            data = all_historical_data.get(ticker, pd.DataFrame())
            if not data.empty:
                fig.add_trace(go.Scatter(
                    x=data.index,
                    y=data['Close'],
                    mode='lines',
                    name=f'{ticker} Close Price'
                ))
        fig.update_layout(
            title="Stock Price Comparison",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            legend_title="Tickers"
        )
        return fig
    except:
        return None
```

### ❓ Why?
- Ensures at least a basic line‑by‑line comparison if LLM‑generated script fails.
- Keeps the user experience consistent even under errors.